In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/processed_tweets_binary.csv')

In [3]:
from sklearn.model_selection import train_test_split

X = df["processed_tweet"]
y = df["emotion_encoded"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2483,), (1065,), (2483,), (1065,))

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))

vectorizer = TfidfVectorizer(stop_words="english")
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Convert to dense matrix
X_train_vec_dense = X_train_vec.todense()
X_test_vec_dense = X_test_vec.todense()

# Model
model = Sequential()
model.add(Dense(10, input_dim=X_train_vec_dense.shape[1], activation='relu'))
model.add(Dense(2, activation='sigmoid'))  # Assuming 3 classes

# Compilation
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

# Training
model.fit(X_train_vec_dense, y_train_encoded, epochs=10, batch_size=5)

Epoch 1/10
497/497 [==============================] - 1s 1ms/step - loss: 0.5058 - accuracy: 0.8365
Epoch 2/10
497/497 [==============================] - 1s 1ms/step - loss: 0.3801 - accuracy: 0.8401
Epoch 3/10
497/497 [==============================] - 1s 1ms/step - loss: 0.3022 - accuracy: 0.8598
Epoch 4/10
497/497 [==============================] - 1s 1ms/step - loss: 0.2092 - accuracy: 0.9223
Epoch 5/10
497/497 [==============================] - 1s 1ms/step - loss: 0.1455 - accuracy: 0.9577
Epoch 6/10
497/497 [==============================] - 1s 1ms/step - loss: 0.1046 - accuracy: 0.9742
Epoch 7/10
497/497 [==============================] - 1s 2ms/step - loss: 0.0774 - accuracy: 0.9835
Epoch 8/10
497/497 [==============================] - 1s 1ms/step - loss: 0.0595 - accuracy: 0.9907
Epoch 9/10
497/497 [==============================] - 1s 1ms/step - loss: 0.0467 - accuracy: 0.9915
Epoch 10/10
497/497 [==============================] - 1s 1ms/step - loss: 0.0373 - accuracy: 0.9952

In [5]:
from sklearn.metrics import classification_report

preds = np.argmax(model.predict(X_test_vec_dense), axis=1)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.68      0.44      0.53       173
           1       0.90      0.96      0.93       892

    accuracy                           0.88      1065
   macro avg       0.79      0.70      0.73      1065
weighted avg       0.86      0.88      0.86      1065



,Unnamed: 0,unprocessed_tweet,product,emotion,processed_tweet,emotion_encoded
24,27,someone started an #austin @PartnerHub group i...,Other Google product or service,Positive emotion,someone start austin partnerhub group google g...,1
42,69,Check out @mention @mention &amp; @mention in ...,iPad,Positive emotion,check amp line ipad austin power sxswi,1
50,89,Thanks to @mention for her mention of our new ...,iPad or iPhone App,Positive emotion,thanks new speech ipad apps showcased conf sxs...,1
55,101,Headline: &quot;#iPad 2 is the Must-Have Gadge...,iPad,Positive emotion,headline quotipad musthave gadget sxswquot hmm...,1
62,114,"#SXSW and #Apple iPad 2's are great, but thoug...",iPad,Positive emotion,apple ipad great thought japan apac region dea...,1
...,...,...,...,...,...,...
3522,9022,"By the way, I love that y'all are so down to d...",Apple,Positive emotion,way love yall discus applebreeds midst presxsw...,1
3525,9029,"[TOP STORY] At #SXSW, Apple schools the market...",Apple,Positive emotion,top story apple school marketing expert thousa...,1
3538,9063,@mention You should get the iPad 2 to save yo...,iPad,Positive emotion,get ipad save back lug laptop sxswmymistake,1
3542,9072,@mention your iPhone 4 cases are Rad and Ready...,iPhone,Positive emotion,iphone case rad ready stop tomorrow get zazzle...,1
